In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/2020-Move/Learning/DS4A-correlation-one/DS4A Project/repo/data/'
# We can combine the datasets into one DF here and begin our analyses

In [5]:
import pandas as pd
import numpy as np
!pip install datetime-quarter
from datequarter import DateQuarter

Names of files to be merged:

Wildfires:
fires_by_county.csv and fires_by_zip.csv

Airports:
airport_data_aggregates.csv

Airbnb:
Availability_by_City.csv,
Availability_by_Zipcode.csv,
(to be added) Availability_by_County.csv

AQI:
BA_AQI_counties.csv


Columns:

Date (datetime) and county or zipcode used to merge Airbnb, AQI, and wildfires

Date (quarters) can be used to append airport data after the prior 3 datasets

In [6]:
#drive path
path = '/content/drive/MyDrive/2020-Move/Learning/DS4A-correlation-one/DS4A Project/repo/data/'

# reading all of the datasets from the data folder
df_airbnb_city = pd.read_csv(path+'Availabitlity_by_City.csv')
df_airbnb_zip = pd.read_csv(path+'Availabitlity_by_Zipcode.csv')
df_airport = pd.read_csv(path+'airport_data_aggregates.csv')
df_aqi_county = pd.read_csv(path+'BA_AQI_COUNTIES.csv')
df_aqi = pd.read_csv(path+'BA_AQI_ZIP_CODES.csv')
df_fires_county = pd.read_csv(path+'fires_by_county.csv')
df_fires_zip = pd.read_csv(path+'fires_by_zip.csv')
df_mobility = pd.read_csv(path+'BA_County_Mobility_2020.csv')

FileNotFoundError: ignored

In [ ]:
def see_df(df):
  print('Data dimensions:', df.shape, '\n')
  return df.head(2)

In [ ]:
see_df(df_airbnb_city)

Data dimensions: (36847, 3) 



,City,Date,Value
0,00000,2015-05-04,0.666667
1,00000,2015-05-05,0.545455


In [ ]:
see_df(df_airbnb_zip)

Data dimensions: (210620, 3) 



,Zipcode,Date,Value
0,0,2015-06-29,NaN
1,0,2015-06-30,NaN


In [ ]:
see_df(df_airport) #update the csv. still showing older sheet

Data dimensions: (33564, 12) 



,Unnamed: 0,QQ-YYYY,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,PASSENGERS,MEDIAN_ITIN_FARE,AVG_ITIN_FARE,MIN_ITIN_FARE,MAX_ITIN_FARE,ORIGIN_CITY_NAME,ORIGIN_LATITUDE,ORIGIN_LONGITUDE
0,0,Q4 of 2010,10135,13796,1.0,10.0,10.000000,10.0,10.0,"Allentown/Bethlehem/Easton, PA",40.654722,-75.438333
1,1,Q4 of 2010,10135,14771,59.0,466.0,457.350877,10.0,1059.0,"Allentown/Bethlehem/Easton, PA",40.654722,-75.438333


In [ ]:
df_aqi

NameError: ignored

In [ ]:
see_df(df_fires_county)

Data dimensions: (17440, 4) 



,Unnamed: 0,Date,Active_wildfires,County
0,0,2015-01-01,False,Alameda
1,1,2015-01-02,False,Alameda


In [ ]:
see_df(df_fires_zip)

Data dimensions: (664900, 4) 



,Unnamed: 0,Date,Active_wildfires,Zipcode
0,0,2015-01-01,False,94002
1,1,2015-01-02,False,94002


In [ ]:
see_df(df_mobility)

Data dimensions: (2709, 8) 



,COUNTY,DATE,RETAIL/RECREATION,GROCERY/PHARMACY,PARKS,TRANSIT/STATIONS,WORKPLACES,RESIDENTIAL
0,Alameda,2020-02-15,1.0,0.0,22.0,1.0,0.0,-1.0
1,Alameda,2020-02-16,7.0,0.0,24.0,5.0,1.0,-2.0
